In [1]:
from distutils.dir_util import copy_tree

# copy subdirectory example
fromDirectory = "FRUIT-16K-PREPROCESSED/test"
toDirectory = "FRUIT-16K-PREPROCESSED/new_test"

copy_tree(fromDirectory, toDirectory)

['FRUIT-16K-PREPROCESSED/new_test\\F_Banana\\1.jpg',
 'FRUIT-16K-PREPROCESSED/new_test\\F_Banana\\1000.jpg',
 'FRUIT-16K-PREPROCESSED/new_test\\F_Banana\\105.jpg',
 'FRUIT-16K-PREPROCESSED/new_test\\F_Banana\\110.jpg',
 'FRUIT-16K-PREPROCESSED/new_test\\F_Banana\\111.jpg',
 'FRUIT-16K-PREPROCESSED/new_test\\F_Banana\\115.jpg',
 'FRUIT-16K-PREPROCESSED/new_test\\F_Banana\\122.jpg',
 'FRUIT-16K-PREPROCESSED/new_test\\F_Banana\\126.jpg',
 'FRUIT-16K-PREPROCESSED/new_test\\F_Banana\\128.jpg',
 'FRUIT-16K-PREPROCESSED/new_test\\F_Banana\\13.jpg',
 'FRUIT-16K-PREPROCESSED/new_test\\F_Banana\\135.jpg',
 'FRUIT-16K-PREPROCESSED/new_test\\F_Banana\\14.jpg',
 'FRUIT-16K-PREPROCESSED/new_test\\F_Banana\\140.jpg',
 'FRUIT-16K-PREPROCESSED/new_test\\F_Banana\\143.jpg',
 'FRUIT-16K-PREPROCESSED/new_test\\F_Banana\\153.jpg',
 'FRUIT-16K-PREPROCESSED/new_test\\F_Banana\\155.jpg',
 'FRUIT-16K-PREPROCESSED/new_test\\F_Banana\\161.jpg',
 'FRUIT-16K-PREPROCESSED/new_test\\F_Banana\\17.jpg',
 'FRUIT-16K-PR

In [4]:
import os
import shutil

source1 = "FRUIT-16K-PREPROCESSED/new_test"
dest1 = "FRUIT-16K-PREPROCESSED/new_valid"

for dir in os.listdir(source1):
  if not os.path.exists(os.path.join(dest1, dir)):
    os.makedirs(os.path.join(dest1, dir))

  files = os.listdir(os.path.join(source1, dir))

  for f in files[0:int(len(files)/2)]:
    shutil.move(source1 + '/' + dir + '/' + f, dest1 + '/' + dir + '/' + f)

In [9]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set the seed
tf.random.set_seed(42)

# Preprocess data (get all of the pixel values between 1 and 0, also called scaling/normalization)
train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Setup the train and test directories
train_dir = "FRUIT-16K-PREPROCESSED/train/" # 80% of FRUIT-16K
valid_dir = "FRUIT-16K-PREPROCESSED/new_valid/" # 10% of FRUIT-16K
test_dir = "FRUIT-16K-PREPROCESSED/new_test/" # 10% of FRUIT-16K

# Import data from directories and turn it into batches
train_data = train_datagen.flow_from_directory(train_dir,
                                               batch_size=32,
                                               target_size=(224, 224),
                                               class_mode="categorical",
                                               seed=42)

valid_data = valid_datagen.flow_from_directory(valid_dir,
                                               batch_size=32,
                                               target_size=(224, 224),
                                               class_mode="categorical",
                                               seed=42)
                                               
test_data  = test_datagen.flow_from_directory(test_dir,
                                               batch_size=32,
                                               target_size=(224, 224),
                                               class_mode="categorical",
                                               seed=42)

Found 12800 images belonging to 16 classes.
Found 1598 images belonging to 16 classes.
Found 1602 images belonging to 16 classes.


In [10]:
checkpoint_filepath = './model_5_callbacks'
model_5_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [11]:
base_model = tf.keras.applications.xception.Xception(include_top=False, weights='imagenet')
base_model.trainable = False

inputs = tf.keras.layers.Input(shape=(224, 224, 3), name="input_layer")

x = base_model(inputs)
print(f"Shape after base_model: {x.shape}")

x = tf.keras.layers.GlobalAveragePooling2D(name="global_average_pooling_layer")(x)
print(f"After GlobalAveragePooling2D(): {x.shape}")

outputs = tf.keras.layers.Dense(16, activation="softmax", name="output_layer")(x)

model_5 = tf.keras.Model(inputs, outputs)

model_5.compile(loss="categorical_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

history_5 = model_5.fit(train_data,
                        epochs=5,
                        validation_data=valid_data,
                        validation_steps=len(valid_data),
                        callbacks=[model_5_checkpoint_callback])

Shape after base_model: (None, 7, 7, 2048)
After GlobalAveragePooling2D(): (None, 2048)
Epoch 1/5
400/400 [==============================] - 28s 64ms/step - loss: 0.3472 - accuracy: 0.9270 - val_loss: 0.0923 - val_accuracy: 0.9862
Epoch 2/5
400/400 [==============================] - 24s 61ms/step - loss: 0.0697 - accuracy: 0.9905 - val_loss: 0.0509 - val_accuracy: 0.9925
Epoch 3/5
400/400 [==============================] - 24s 60ms/step - loss: 0.0384 - accuracy: 0.9953 - val_loss: 0.0370 - val_accuracy: 0.9944
Epoch 4/5
400/400 [==============================] - 25s 61ms/step - loss: 0.0257 - accuracy: 0.9975 - val_loss: 0.0287 - val_accuracy: 0.9981
Epoch 5/5
400/400 [==============================] - 24s 61ms/step - loss: 0.0177 - accuracy: 0.9984 - val_loss: 0.0242 - val_accuracy: 0.9981


In [12]:
model_5.evaluate(test_data)

51/51 [==============================] - 3s 54ms/step - loss: 0.0309 - accuracy: 0.9938


[0.030876826494932175, 0.9937577843666077]